In [48]:
import pandas as pd
import sys
import os

# Configurar el path para poder importar tools
# El notebook está en ENTREGABLES/marcos/, necesitamos subir un nivel a ENTREGABLES/
current_dir = os.getcwd()
# Si estamos en marcos/, subir un nivel
if current_dir.endswith('marcos'):
    entregables_dir = os.path.dirname(current_dir)
else:
    # Si no, buscar el directorio ENTREGABLES en el path actual
    entregables_dir = current_dir
    while not os.path.exists(os.path.join(entregables_dir, 'tools')):
        parent = os.path.dirname(entregables_dir)
        if parent == entregables_dir:  # Llegamos a la raíz
            break
        entregables_dir = parent

if os.path.exists(os.path.join(entregables_dir, 'tools')):
    if entregables_dir not in sys.path:
        sys.path.insert(0, entregables_dir)
    print(f"✅ Path configurado: {entregables_dir}")
else:
    print("⚠️ No se encontró el directorio tools. Verifica la estructura del proyecto.")

# Importar la función seleccionar_variables
from tools import seleccionar_variables



⚠️ No se encontró el directorio tools. Verifica la estructura del proyecto.


In [56]:
df = pd.read_csv("../../DATA/processed/fraud_dataset_processed.csv")

df = seleccionar_variables(df, o_dummies=True, pais_encoding=True)

In [57]:
pd.set_option('display.max_columns', None)

df.head(100)

,a,b,c,d,e,f,h,k,l,m,n,q,r,s,monto,score,o_is_Y,o_is_N,o_is_NA,categoria_id,pais_target_enc,pais_freq_enc,producto_nombre
0,0.8,0.7685,0.006804,0.403846,0.000534,0.000041,0.100000,0.883602,0.031942,0.046271,1.0,0.411765,0.006804,0.0,0.001520,0.66,0.0,0.0,1.0,cat_8d714cd,0.033238,1.000000,Máquininha Corta Barba Cabelo Peito Perna Pelo...
1,0.8,0.7550,0.000667,0.038462,0.000000,0.000262,0.016667,0.376018,0.531345,0.000449,1.0,0.039216,0.000667,0.0,0.033733,0.72,1.0,0.0,0.0,cat_64b574b,0.033238,1.000000,Avental Descartavel Manga Longa - 50 Un. Tnt ...
2,0.8,0.7455,0.017476,0.076923,0.000000,0.000165,0.400000,0.516369,0.235918,0.035040,1.0,0.078431,0.017476,0.0,0.091794,0.95,0.0,0.0,1.0,cat_e9110c5,0.021982,0.288501,Bicicleta Mountain Fire Bird Rodado 29 Alumini...
3,0.8,0.7631,0.001364,0.980769,0.000579,0.000158,0.400000,0.154034,0.225977,0.515723,1.0,0.980392,0.001363,1.0,0.000952,0.02,0.0,0.0,1.0,cat_d06e653,0.033238,1.000000,Caneta Delineador Carimbo Olho Gatinho Longo 2...
4,0.4,0.7315,0.000413,0.307692,0.000000,0.000041,0.050000,0.855802,0.135984,0.067835,1.0,0.294118,0.000413,0.0,0.000950,0.76,0.0,0.0,1.0,cat_6c4cfdc,0.033238,1.000000,Resident Evil Operation Raccoon City Ps3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.8,0.7861,0.013281,0.980769,0.000286,0.000048,0.533333,0.047984,0.044135,0.147799,1.0,1.000000,0.013281,1.0,0.002581,0.13,0.0,0.0,1.0,cat_430bfa8,0.033238,1.000000,Kit Sobrevivencia Bússola Profissional + Peder...
96,0.8,0.6845,0.019387,0.038462,0.000000,0.000048,0.166667,0.102148,0.784228,0.057502,1.0,0.039216,0.019387,0.0,0.015832,0.38,0.0,0.0,1.0,cat_41d2590,0.021982,0.288501,Campera Ansilta Pampa Evo Polartec Mujer
97,0.4,0.4007,0.000697,0.057692,0.000000,0.000083,0.016667,0.493420,0.022399,0.001348,0.0,0.058824,0.000697,0.0,0.002048,0.90,0.0,0.0,1.0,cat_d5975ae,0.021982,0.288501,God Of War Collection Ps3 Digital Audio Españo...
98,0.8,0.6697,0.000883,0.365385,0.000000,0.000041,0.050000,0.170332,0.259775,0.032345,1.0,0.352941,0.000883,0.0,0.006834,0.02,0.0,0.0,1.0,cat_df60aa8,0.033238,1.000000,Kit Teclado+mouse S/fio Slim K-w510 Multimídia...


In [21]:
num_na_pais = df['pais'].isna().sum()
print(f"Cantidad de NAs en 'pais': {num_na_pais}")



Cantidad de NAs en 'pais': 0
